# 1. Introduction
`LangChain` is a framework for developing applications powered by language models. It enables applications that are:

`Data-aware:` connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

`Components:` abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.

# 2. Installation

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install langchain

# 3. Basics

In [30]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [ ]:
# Create .env file and write this line [your OPENAI key]
# You will get OPENAI key from 
# platform.openai.com -> Personal -> view API keys -> create new one & copy paste in .env 
# OPENAI_API_KEY="sk-rkqECXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" 

In [34]:
load_dotenv(find_dotenv()) # This will return True if successfully loaded OPENAI KEY
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key

In [ ]:
# If above code not loading your key then use this.
def loadKey():
    my_key = open('.env').read().split('=')[1][1:-1]
    os.environ["OPENAI_API_KEY"] = my_key
    openai.api_key = my_key
loadKey()

In [37]:
# Lets create a simple Chat-Response function

def chat(input):
    messages = [{"role": "user", "content": input}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
    )
    return response

In [39]:
output = chat("What is the capital of India")
output.choices[0].message["content"]

'The capital of India is New Delhi.'

In [53]:
question = "What is the capital of India"

prompt = """
Be very funny when answering questions
Question: {question}
""".format(question=question)

print(prompt)
chat(prompt).choices[0].message['content']


Be very funny when answering questions
Question: What is the capital of India



"Oh, that's easy! The capital of India is... wait for it... drumroll please... New York City! Just kidding, it's actually New Delhi. But wouldn't it be funny if it was New York City? Imagine all the confused tourists!"

# 4. Prompt.

Prompt Templates makes creating a prompt for different usecases easier than using f-string.

In [55]:
import langchain as lc

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# You can see we don't require to create message for OpenAI.ChatCompletion
llms = lc.llms.OpenAI(model_name = 'gpt-3.5-turbo')

# Both are correct.
#llms('What is capital of India?')
llms.predict('what is the capital of india?')

In [63]:
# Here we are creating a prompt similar kind we creat in chatGPT.
# We are expecting output in JSON format with 2 keys.
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [66]:
prompt_template = lc.prompts.ChatPromptTemplate.from_template(template = template)
chain = lc.chains.LLMChain(llm=llms, prompt=prompt_template)
chain.predict(input='I order pizza salami and it was awesome!')

'{\n    "sentiment": "positive",\n    "subject": "pizza"\n}'

# 5. Parser, ResponseSchema, Templates, Chains & OutputParsers
We can see, we get desired output. But in string format. Let's convert it into much better way.

In [67]:
sentiment_schema = lc.output_parsers.ResponseSchema(
    name="sentiment",
    description="Is the text positive, neutral or negative? Only provide these words",
)
subject_schema = lc.output_parsers.ResponseSchema(
    name="subject", 
    description="What subject is the text about? Use exactly one word."
)
price_schema = lc.output_parsers.ResponseSchema(
    name="price",
    description="How expensive was the product? Use None if no price was provided in the text",
)

response_schemas = [sentiment_schema, subject_schema, price_schema]

In [69]:
# We don't require to typing every instruction into chat.
# StructuredOutputParser will do for us.
parser = lc.output_parsers.StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text positive, neutral or negative? Only provide these words\n\t"subject": string  // What subject is the text about? Use exactly one word.\n\t"price": string  // How expensive was the product? Use None if no price was provided in the text\n}\n```'

In [70]:
# Now we have instruction read.
# let use with previous template

template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: {input}

{format_instructions}

"""

prompt = lc.prompts.ChatPromptTemplate.from_template(template=template)

# String formatting
messages = prompt.format_messages(
    input="I ordered Pizza Salami for 9.99$ and it was awesome!",
    format_instructions=format_instructions)

chat = lc.chat_models.ChatOpenAI(temperature=0.0)
response = chat(messages)
response

AIMessage(content='```json\n{\n\t"sentiment": "positive",\n\t"subject": "pizza",\n\t"price": "9.99$"\n}\n```', additional_kwargs={}, example=False)

In [71]:
output_dict = parser.parse(response.content)
output_dict

{'sentiment': 'positive', 'subject': 'pizza', 'price': '9.99$'}

In [72]:
# You can see. By just providing text input. we get,
# sentiment, subject, price, without specifying in chat
# what kind of output we want.

# ------------ END ------------

In [1]:
import os
import openai

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv, find_dotenv